In [4]:
from datetime import datetime

a = datetime.now() 
a
a-a

datetime.timedelta(0)

In [1]:
from nlptext.token import Token
from nlptext.utils.grain import generate_batch, getTokenBatchInfo

from nlptext.base import BasicObject

from datetime import datetime


CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True, 'Max_Ngram': 1,},
    'char':    {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'basic':   {'use': True,'Max_Ngram': 2, 'end_grain': False},
    'medical': {'use': True,'Max_Ngram': 2, 'end_grain': False},
    'radical': {'use': True,'Max_Ngram': 2, 'end_grain': False},
    'subcomp': {'use': True,'Max_Ngram': 3, 'end_grain': True},
    'stroke':  {'use': True,'Max_Ngram': 5, 'end_grain': True},
    # CTX_DEP
    'pos':     {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
    # ANNO
    'annoR':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
    'annoE':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
}

In [2]:
BOB = 'channel/WikiTotal/word/Token447170/Pyramid'
LGU = 'channel/WikiTotal/word/Token447170/GrainUnique'
BasicObject.INIT_FROM_PICKLE(BOB, LGU)

BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE)

CORPUS	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
FOLDER	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/FOLDER.p
FOLDER	the length of it is   : 1
TEXT	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/TEXT.p
TEXT	the length of it is   : 4717592
SENT	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/SENT.p
SENT	the length of it is   : 11199643
TOKEN	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/TOKEN.p
TOKEN	the length of it is   : 257789077
**************************************** 

token	read from pickle file : channel/WikiTotal/word/Token447170/GrainUnique/token.p
token	the length of it is   : 447170
**************************************** 

Deal with the Channel: token
Current Channel is        	 token
Current Channel Max_Ngram 	 1
Deal with the Channel: char
Current Channel is        	 char
Current Channel Max_Ngram 	 1
Deal with the Channel:

In [3]:
CH = BasicObject.CHANNEL_SETTINGS
CH

{'token': {'Max_Ngram': 1},
 'char': {'Max_Ngram': 1, 'end_grain': False},
 'basic': {'Max_Ngram': 2, 'end_grain': False},
 'medical': {'Max_Ngram': 2, 'end_grain': False},
 'radical': {'Max_Ngram': 2, 'end_grain': False},
 'subcomp': {'Max_Ngram': 3, 'end_grain': True},
 'stroke': {'Max_Ngram': 5, 'end_grain': True}}

In [54]:
CH = {'token': {'Max_Ngram': 1},
 'char': {'Max_Ngram': 1, 'end_grain': False},
 'basic': {'Max_Ngram': 2, 'end_grain': False},
 'medical': {'Max_Ngram': 2, 'end_grain': False},
 'radical': {'Max_Ngram': 2, 'end_grain': False},
 'subcomp': {'Max_Ngram': 3, 'end_grain': True},
 'stroke': {'Max_Ngram': 5, 'end_grain': True},
}

In [55]:
def test_speed(total_token_num):
    tokenIdx = list(range(total_token_num))
    total_start = datetime.now()
    for ch, cs in CH.items():
        print(ch)
        s_ch = datetime.now()
        for i in tokenIdx:
            tk =Token(i)
            tk.getGrainTensor(ch, **cs, dontUseLookUp=False)
        e_ch = datetime.now()
        print(e_ch - s_ch)

    total_end = datetime.now()
    time = total_end- total_start
    print('Total time:', time)
    return time

In [85]:

total_token_num = 10000
test_speed(total_token_num)

token
0:00:00.018738
char
0:00:00.025603
basic
0:00:00.021415
medical
0:00:00.021506
radical
0:00:00.021772
subcomp
0:00:00.021976
stroke
0:00:00.022232
Total time: 0:00:00.153855


datetime.timedelta(microseconds=153855)

In [106]:
total_token_num = 5000
test_speed(total_token_num)

token
0:00:00.004752
char
0:00:00.009822
basic
0:00:00.013825
medical
0:00:00.011008
radical
0:00:00.012198
subcomp
0:00:00.011416
stroke
0:00:00.011544
Total time: 0:00:00.075161


datetime.timedelta(microseconds=75161)

In [119]:
tk.CONTEXT_IND_CHANNELS

['basic', 'medical', 'radical', 'token', 'char', 'subcomp', 'stroke']

In [118]:
from nlptext.utils.channel import getChannelName

tokenIdx = list(range(10000))
dontUseLookUp = False
TokenNum_Dir = False
UNK_ID = 3
total_start = datetime.now() # <---
self = BasicObject

for ch, cs in CH.items():
    print(ch)
    s_ch = datetime.now()
    for i in tokenIdx:
        tk =Token(i)
        
        if ch == 'token':
            [tk.index], 1
        
        else:
            channel_name = getChannelName(ch,**cs)

            if not dontUseLookUp:
                LOOKUP, TokenUnqiue = self.getLookUp(channel_name = channel_name, TokenNum_Dir = TokenNum_Dir)
                LTU, DTU = TokenUnqiue
                index = DTU.get(tk.token, UNK_ID) if TokenNum_Dir else tk.index
                if index != UNK_ID:
                    info = LOOKUP[index] # TODO, to do what?
                    leng = len(info)

            else:
                LGU, DGU = self.getGrainUnique(channel_name = channel_name, TokenNum_Dir =TokenNum_Dir)
                info = [DGU.get(i, UNK_ID) for i in tk.getChannelGrain(ch, **cs)]
                leng = len(info)
                
    e_ch = datetime.now()
    print(e_ch - s_ch)
    
total_end = datetime.now()
print('Total time:', total_end- total_start)

token
0:00:00.008516
char
0:00:00.020361
basic
0:00:00.021669
medical
0:00:00.020693
radical
0:00:00.020963
subcomp
0:00:00.020903
stroke
0:00:00.020784
Total time: 0:00:00.135016


```
char
0:00:00.009325
basic
0:00:00.008865
medical
0:00:00.009747
radical
0:00:00.007804
subcomp
0:00:00.007518
stroke
0:00:00.007718
token
0:00:00.003510
Total time: 0:00:00.055877
```

In [63]:
# total_token_num = 10000
# result = {str(total_token_num): float(str(test_speed(total_token_num)).split(':')[-1]) for total_token_num in [1000, 2000, 4000, 8000]}

In [20]:
cls =BasicObject

In [24]:
s = datetime.now()
for i in tokenIdx:
    try:
        cls.LOOKUP[TokenNum_Dir] = cls.LOOKUP[TokenNum_Dir] if TokenNum_Dir in cls.LOOKUP else {}
    except:
        pass
e = datetime.now()
print(e-s)

0:00:00.001602


In [37]:
from datetime import datetime
TVC_left, TVC_right = TVC['left'], TVC['right']
from nlptext.utils.channel import getChannelName
from datetime import datetime
import os
UNK_ID = 3


self = BasicObject
# time_per_100_batch = datetime.now()
for step in range(1, 10000+1):
    batch_time_start = datetime.now()#################################################
    
    load_batch_time_start = batch_time_start

    # Step1: generate batch data
    batch_left, batch_right, data_index = generate_batch(tokenList, data_index,  **BATCH)
    # batch_left = [Token(tk_idx) for tk_idx in batch_left]
    
    s_t = datetime.now()
#     batch_Left_Channels_Info_Leng = {ch: getTokenBatchInfo(batch_left, ch, **cs) for ch, cs in CH.items()}

    for ch, cs in CH.items():
        print(ch)
        s_ch = datetime.now()
        s = s_ch
        
        Info_Leng = []
        for index in batch_left:
            channel_name = getChannelName(ch, **cs)
            TokenNum_Dir = cs.get('TokenNum_Dir')
            Max_Ngram =  cs.get('Max_Ngram')
            end_grain=  cs.get('end_grain')
            tagScheme =  cs.get('tagScheme', 'BIO')
            tmp_TokenNum_Dir = TokenNum_Dir if TokenNum_Dir else self.TokenNum_Dir
            lookup_channel_name_path = os.path.join(tmp_TokenNum_Dir, 'LookUp', channel_name + '.p')
            if os.path.isfile(lookup_channel_name_path):
                ### case 1: deal with the case: CTX_IND channel and this channel has LOOKUP.p
                LOOKUP, TokenUnqiue = self.getLookUp(ch, Max_Ngram, end_grain = end_grain, 
                                                     tagScheme = tagScheme, TokenNum_Dir = TokenNum_Dir)

                LTU, DTU = TokenUnqiue
                # index = 9

                if index != UNK_ID:
                    # print('\tUse LookUp from:'+lookup_channel_name_path)
                    info = LOOKUP[index] # TODO, to do what?
                    # info = np.array(info)
                    leng = np.array(len(info), dtype='float32')
                    # leng = len(info)
            Info_Leng.append((info, leng))
        e = datetime.now()
        print('\t', e - s)
        
        s = datetime.now()
        # channelLeng = [info_leng[1] for info_leng in Info_Leng] # little
        # channelLeng = np.array(channelLeng)
        channelLeng = np.array([info_leng[1] for info_leng in Info_Leng], dtype='float32')
        
        maxGrainLeng = int(np.max(channelLeng))
        channelInfo = [info_leng[0] for info_leng in Info_Leng]
        e = datetime.now()
        print('\t', e - s)
        
        s = datetime.now()
#         channelInfo = np.array([np.pad(info,[0,maxGrainLeng - len(info)], 'constant') for info in channelInfo])
        channelInfo_Final =np.zeros([len(channelInfo), maxGrainLeng], dtype=int)
        for idx, info in enumerate(channelInfo):
            channelInfo_Final[idx, :len(info)] = channelInfo[idx]
        e = datetime.now()
        print('\t', e - s)
        
        e_ch = e
        print('\t', ch, e_ch - s_ch)
    
    
    e_t = datetime.now()
    print('\t\t', e_t - s_t)
    
    # batch_Left_Channels_Info = {ch:v[0] for ch, v in batch_Left_Channels_Info_Leng.items()}
    # batch_Left_Channels_Leng = {ch:v[1] for ch, v in batch_Left_Channels_Info_Leng.items()}\
    
    
    batch_right = [Token(tk_idx) for tk_idx in batch_right]
    
    batch_Right_Channels_Info_Leng = {'token': getTokenBatchInfo(batch_right, 'token')}
    batch_Right_Channels_Info = {ch:v[0] for ch, v in batch_Right_Channels_Info_Leng.items()}
    batch_Right_Channels_Leng = {ch:v[1] for ch, v in batch_Right_Channels_Info_Leng.items()}
    
    load_batch_time_end = datetime.now()
    print(load_batch_time_end - load_batch_time_start)#################################################

char
	 0:00:00.111344
	 0:00:00.002255
	 0:00:00.010229
	 char 0:00:00.124160
basic
	 0:00:00.091620
	 0:00:00.003841
	 0:00:00.014422
	 basic 0:00:00.110083
medical
	 0:00:00.113518
	 0:00:00.001496
	 0:00:00.010786
	 medical 0:00:00.125986
radical
	 0:00:00.112859
	 0:00:00.002941
	 0:00:00.010814
	 radical 0:00:00.126755
subcomp
	 0:00:00.099234
	 0:00:00.001936
	 0:00:00.021576
	 subcomp 0:00:00.122867
stroke
	 0:00:00.102684
	 0:00:00.002610
	 0:00:00.063201
	 stroke 0:00:00.168635
		 0:00:00.779424
0:00:00.903077
char
	 0:00:00.101039
	 0:00:00.002171
	 0:00:00.011241
	 char 0:00:00.114557
basic
	 0:00:00.096462
	 0:00:00.001898
	 0:00:00.009872
	 basic 0:00:00.108328
medical
	 0:00:00.118875
	 0:00:00.003361
	 0:00:00.019948
	 medical 0:00:00.142637
radical
	 0:00:00.123382
	 0:00:00.002507
	 0:00:00.010382
	 radical 0:00:00.136382
subcomp
	 0:00:00.117160
	 0:00:00.002497
	 0:00:00.021940
	 subcomp 0:00:00.141803
stroke
	 0:00:00.121830
	 0:00:00.002785
	 0:00:00.055707
	 strok

KeyboardInterrupt: 

In [66]:
# from datetime import datetime
# TVC_left, TVC_right = TVC['left'], TVC['right']
# from nlptext.utils.channel import getChannelName
# from datetime import datetime
# import os
# UNK_ID = 3


# self = BasicObject
# # time_per_100_batch = datetime.now()
# for step in range(1, 10000+1):
#     batch_time_start = datetime.now()#################################################
    
#     load_batch_time_start = batch_time_start

#     # Step1: generate batch data
#     batch_left, batch_right, data_index = generate_batch(tokenList, data_index,  **BATCH)
#     # batch_left = [Token(tk_idx) for tk_idx in batch_left]
    
#     s_t = datetime.now()
# #     batch_Left_Channels_Info_Leng = {ch: getTokenBatchInfo(batch_left, ch, **cs) for ch, cs in CH.items()}

#     for ch, cs in CH.items():
#         print(ch)
#         s_ch = datetime.now()
#         s = s_ch
        
#         Info_Leng = []
#         for index in batch_left:
#             channel_name = getChannelName(ch, **cs)
#             TokenNum_Dir = cs.get('TokenNum_Dir')
#             Max_Ngram =  cs.get('Max_Ngram')
#             end_grain=  cs.get('end_grain')
#             tagScheme =  cs.get('tagScheme', 'BIO')
#             tmp_TokenNum_Dir = TokenNum_Dir if TokenNum_Dir else self.TokenNum_Dir
#             lookup_channel_name_path = os.path.join(tmp_TokenNum_Dir, 'LookUp', channel_name + '.p')
#             if os.path.isfile(lookup_channel_name_path):
#                 ### case 1: deal with the case: CTX_IND channel and this channel has LOOKUP.p
#                 LOOKUP, TokenUnqiue = self.getLookUp(ch, Max_Ngram, end_grain = end_grain, 
#                                                      tagScheme = tagScheme, TokenNum_Dir = TokenNum_Dir)

#                 LTU, DTU = TokenUnqiue
#                 # index = 9

#                 if index != UNK_ID:
#                     # print('\tUse LookUp from:'+lookup_channel_name_path)
#                     info = LOOKUP[index] # TODO, to do what?
#                     # info = np.array(info)
#                     # leng = np.array(len(info), dtype='float32')
#                     leng = len(info)
#             Info_Leng.append((info, leng))
#         e = datetime.now()
#         print('\t', e - s)
        
#         s = datetime.now()
#         # channelLeng = [info_leng[1] for info_leng in Info_Leng] # little
#         # channelLeng = np.array(channelLeng)
#         channelLeng = np.array([info_leng[1] for info_leng in Info_Leng], dtype='float32')
        
#         maxGrainLeng = int(np.max(channelLeng))
#         channelInfo = [info_leng[0] for info_leng in Info_Leng]
#         e = datetime.now()
#         print('\t', e - s)
        
#         s = datetime.now()
# #         channelInfo = np.array([np.pad(info,[0,maxGrainLeng - len(info)], 'constant') for info in channelInfo])
#         channelInfo_Final =np.zeros([len(channelInfo), maxGrainLeng], dtype=int)
#         for idx, info in enumerate(channelInfo):
#             channelInfo_Final[idx, :len(info)] = channelInfo[idx]
#         e = datetime.now()
#         print('\t', e - s)
        
#         e_ch = e
#         print('\t', ch, e_ch - s_ch)
    
    
#     e_t = datetime.now()
#     print('\t\t', e_t - s_t)
    
#     # batch_Left_Channels_Info = {ch:v[0] for ch, v in batch_Left_Channels_Info_Leng.items()}
#     # batch_Left_Channels_Leng = {ch:v[1] for ch, v in batch_Left_Channels_Info_Leng.items()}\
    
    
#     s_t = datetime.now()
    
#     batch_right = [Token(tk_idx) for tk_idx in batch_right]
    
#     batch_Right_Channels_Info_Leng = {'token': getTokenBatchInfo(batch_right, 'token')}
#     batch_Right_Channels_Info = {ch:v[0] for ch, v in batch_Right_Channels_Info_Leng.items()}
#     batch_Right_Channels_Leng = {ch:v[1] for ch, v in batch_Right_Channels_Info_Leng.items()}
#     e_t = datetime.now()
#     print('\t\t', e_t - s_t)
    
    
#     load_batch_time_end = datetime.now()
#     print(load_batch_time_end - load_batch_time_start)#################################################

In [1]:
from nlptext.token import Token
from nlptext.utils.grain import generate_batch, getTokenBatchInfo

from config_embed import CHANNEL_SETTINGS_TEMPLATE, TK_VEC_CHANNEL_SETTINGS_TEMPLATE
from config_embed import MERGING_SETTINGS_TEMPLATE, BATCH_SETTINGS_TEMPLATE
from config_embed import EMBED_TRAINING_SETTINGS_TEMPLATE

from nlptext.base import BasicObject

from nlpembed.config_factory import setting_factory
#from nlpembed.train import Train_Embedding
from nlptext.utils.channel import getChannelName
from datetime import datetime

BOB = 'channel/WikiTotal/word/Token447170/Pyramid'
LGU = 'channel/WikiTotal/word/Token447170/GrainUnique'
BasicObject.INIT_FROM_PICKLE(BOB, LGU)



CORPUS	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
FOLDER	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/FOLDER.p
FOLDER	the length of it is   : 1
TEXT	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/TEXT.p
TEXT	the length of it is   : 4717592
SENT	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/SENT.p
SENT	the length of it is   : 11199643
TOKEN	read from pickle file : channel/WikiTotal/word/Token447170/Pyramid/TOKEN.p
TOKEN	the length of it is   : 257789077
**************************************** 

token	read from pickle file : channel/WikiTotal/word/Token447170/GrainUnique/token.p
token	the length of it is   : 447170
**************************************** 



```
# 10000
char
0:00:00.095421
basic
0:00:00.095452
medical
0:00:00.088307
radical
0:00:00.085013
subcomp
0:00:00.088327
stroke
0:00:00.084229
token
0:00:00.122542
Total time: 0:00:00.660324
```

```
# 5000
char
0:00:00.061882
basic
0:00:00.043449
medical
0:00:00.042075
radical
0:00:00.041467
subcomp
0:00:00.045256
stroke
0:00:00.041615
token
0:00:00.062675
Total time: 0:00:00.339320
```

In [122]:

PAD   = '</pad>'
START = '</start>'
END   = '</end>'
UNK   = '</unk>'
specialTokens     = [ PAD, START, END, UNK]


def trans_bioesTag(channel, bioesTag, tagScheme):

    if bioesTag in specialTokens:
        return bioesTag
        
    if 'S' not in tagScheme and 'E' not in tagScheme:
        i = bioesTag.replace('-S', '-B').replace('-E', '-I')
    elif 'S' not in tagScheme:
        i = bioesTag.replace('-S', '-B')
    elif 'E' not in tagScheme:
        i = bioesTag.replace('-E', '-I')
    else:
        i = bioesTag

    if channel == 'annoR':
        return i.split('-')[-1] 
    else:
        return i



In [123]:

def readFile2GrainList(channel_name_path):
    ListGrainUnique = []
    with open(channel_name_path, 'r', encoding = 'utf-8') as f:
        for gr in f.readlines():
            gr = '\n' if  '\\n' in gr[:-1] else gr[:-1]
            ListGrainUnique.append(gr)
    return ListGrainUnique


In [143]:
bioesLGU = readFile2GrainList(channel_name_path='channel/LuohuNER/char/Token1601/annoE-bioes.tsv')

In [144]:
bioesLGU

['</pad>',
 '</start>',
 '</end>',
 'O',
 '主体-B',
 '主体-E',
 '主体-I',
 '主体-S',
 '异常-B',
 '异常-E',
 '异常-I',
 '异常-S',
 '异常分类-B',
 '异常分类-E',
 '异常分类-I',
 '异常分类-S',
 '检查-B',
 '检查-E',
 '检查-I',
 '检查-S',
 '治疗-B',
 '治疗-E',
 '治疗-I',
 '治疗-S']

In [156]:
channel = 'annoR'
tagScheme = 'BIOES'

new_LGU = [trans_bioesTag(channel, i, tagScheme) for i in bioesLGU]

b = list(set(new_LGU[3:]))
b.sort()
new_LGU = new_LGU[:3] + b
new_LGU

['</pad>', '</start>', '</end>', 'B', 'E', 'I', 'O', 'S']

In [157]:

new_DGU = dict(zip(new_LGU, range(len(new_LGU))))
new_DGU

{'</pad>': 0,
 '</start>': 1,
 '</end>': 2,
 'B': 3,
 'E': 4,
 'I': 5,
 'O': 6,
 'S': 7}

In [158]:
 {idx: new_DGU[trans_bioesTag(channel, bioesTag, tagScheme )] 
                                                   for idx, bioesTag in enumerate(bioesLGU)}

{0: 0,
 1: 1,
 2: 2,
 3: 6,
 4: 3,
 5: 4,
 6: 5,
 7: 7,
 8: 3,
 9: 4,
 10: 5,
 11: 7,
 12: 3,
 13: 4,
 14: 5,
 15: 7,
 16: 3,
 17: 4,
 18: 5,
 19: 7,
 20: 3,
 21: 4,
 22: 5,
 23: 7}

In [159]:
 {bioesTag: trans_bioesTag(channel, bioesTag, tagScheme )
                                                   for idx, bioesTag in enumerate(bioesLGU)}

{'</pad>': '</pad>',
 '</start>': '</start>',
 '</end>': '</end>',
 'O': 'O',
 '主体-B': 'B',
 '主体-E': 'E',
 '主体-I': 'I',
 '主体-S': 'S',
 '异常-B': 'B',
 '异常-E': 'E',
 '异常-I': 'I',
 '异常-S': 'S',
 '异常分类-B': 'B',
 '异常分类-E': 'E',
 '异常分类-I': 'I',
 '异常分类-S': 'S',
 '检查-B': 'B',
 '检查-E': 'E',
 '检查-I': 'I',
 '检查-S': 'S',
 '治疗-B': 'B',
 '治疗-E': 'E',
 '治疗-I': 'I',
 '治疗-S': 'S'}

# Test Batch Load Speed

In [1]:

from pprint import pprint
from nlptext.base import BasicObject

CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True, 'Max_Ngram': 1,},
    'char':    {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': True,'Max_Ngram': 3, 'end_grain': True},
    'stroke':  {'use': True,'Max_Ngram': 5, 'end_grain': True},
    # CTX_DEP
    'pos':     {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
    # ANNO
    'annoR':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
    'annoE':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
}

# Path2Pyramid = 'data/boson/char/Token3870/Pyramid/'
# Path2LGUnique = 'data/boson/char/Token3870/GrainUnique/'



Path2Pyramid  = 'data/LuohuCorpus/char/Token3546/Pyramid/'
Path2LGUnique = 'data/LuohuCorpus/char/Token3546/GrainUnique/'



BasicObject.INIT_FROM_PICKLE(Path2Pyramid, Path2LGUnique)

BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE)

CORPUS	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
FOLDER	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/FOLDER.p
FOLDER	the length of it is   : 44
TEXT	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/TEXT.p
TEXT	the length of it is   : 227876
SENT	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/SENT.p
SENT	the length of it is   : 1213151
TOKEN	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/TOKEN.p
TOKEN	the length of it is   : 36760202
**************************************** 

token	read from pickle file : data/LuohuCorpus/char/Token3546/GrainUnique/token.p
token	the length of it is   : 3546
**************************************** 

Deal with the Channel: token
Current Channel is        	 token
Current Channel Max_Ngram 	 1
Deal with the Channel: char
Current Channel is        	 char
Current Channel Max_Ngram 	 1
Deal with the Channel: basic
Current Channel

In [4]:
# from nlptext.utils.grain import generate_batch_idx
from nlpembed.utils.batch import generate_batch_idx

maxlen = BasicObject.TOKEN['length']

BATCH = dict(
    skip_window = 5,
    batch_token_num = 1000,
    maxlen = maxlen,
    left2right = 'tgt-ctx', # 'ctx-tgt', #
    size2size  = '1-1' , # 'All-1' (for ctx-tgt only) and '1-All' (for tgt-ctx only)
    tgtInctx   = False,
    # tgt_start_idx = skip_window,
)

if BATCH['size2size'] == '1-1':
    BATCH['batch_size'] = BATCH['batch_token_num'] * (BATCH['skip_window']*2 + int(BATCH['tgtInctx']))
else:
    BATCH['batch_size'] = BATCH['batch_token_num']
    
    
print(BATCH)
tgt_start_idx = BATCH['skip_window']
left, right, data_index = generate_batch_idx(tgt_start_idx,  **BATCH)

list(zip(left, right))

{'skip_window': 5, 'batch_token_num': 1000, 'maxlen': 36760202, 'left2right': 'tgt-ctx', 'size2size': '1-1', 'tgtInctx': False, 'batch_size': 10000}


[(5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 9),
 (8, 10),
 (8, 11),
 (8, 12),
 (8, 13),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (9, 8),
 (9, 10),
 (9, 11),
 (9, 12),
 (9, 13),
 (9, 14),
 (10, 5),
 (10, 6),
 (10, 7),
 (10, 8),
 (10, 9),
 (10, 11),
 (10, 12),
 (10, 13),
 (10, 14),
 (10, 15),
 (11, 6),
 (11, 7),
 (11, 8),
 (11, 9),
 (11, 10),
 (11, 12),
 (11, 13),
 (11, 14),
 (11, 15),
 (11, 16),
 (12, 7),
 (12, 8),
 (12, 9),
 (12, 10),
 (12, 11),
 (12, 13),
 (12, 14),
 (12, 15),
 (12, 16),
 (12, 17),
 (13, 8),
 (13, 9),
 (13, 10),
 (13, 11),
 (13, 12),
 (13, 14),
 (13, 15),
 (13, 16),
 (13, 17),
 (13, 18),
 (14, 9),
 (14, 10),
 (14, 11),
 (14, 12),
 (14, 13),
 (14, 15),
 (14, 16),
 (14, 17),
 (14, 18),
 (14, 19),


In [5]:
left

array([   5,    5,    5, ..., 1004, 1004, 1004])

In [8]:
CH = {'token': {'Max_Ngram': 1},
 'char': {'Max_Ngram': 1, 'end_grain': False},
 'basic': {'Max_Ngram': 2, 'end_grain': False},
 'medical': {'Max_Ngram': 2, 'end_grain': False},
 'radical': {'Max_Ngram': 2, 'end_grain': False},
 'subcomp': {'Max_Ngram': 3, 'end_grain': True},
 'stroke': {'Max_Ngram': 5, 'end_grain': True},
  'pos': {'Max_Ngram': 1, 'end_grain': False, 'tagScheme': 'BIO'},
     }

channel = 'stroke'
cs = CH[channel]

In [9]:
import numpy as np
from nlptext.token import Token


def transfer_batch(batch_idx):
    if len(batch_idx.shape) == 2:
        elen, tkn = batch_idx.shape
        batch_idx = np.reshape(batch_idx, (elen*tkn))# .shape
    else:
        elen, tkn = batch_idx.shape[0], 1
    batch = np.array([Token(token_idx) for token_idx in batch_idx]).reshape(elen, tkn)
    return batch
    
batch = transfer_batch(left)
elen, tkn = batch.shape
batch = np.reshape(batch, (elen*tkn))# .shape
Info_Leng = [tk.getGrainTensor(channel, **cs)  for tk in batch]
channelLeng = np.array([info_leng[1] for info_leng in Info_Leng], dtype='float32')
maxGrainLeng = int(np.max(channelLeng))

channelInfo = [info_leng[0] for info_leng in Info_Leng]

channelInfo_Final =np.zeros([len(channelInfo), maxGrainLeng], dtype=int)
for idx, info in enumerate(channelInfo):
    channelInfo_Final[idx, :len(info)] = channelInfo[idx]

channelInfo_Final = channelInfo_Final.reshape(elen, tkn, maxGrainLeng)
channelLeng = channelLeng.reshape(elen, tkn)

print(channelInfo_Final.shape, channelLeng.shape)

Get LookUp Table for Channel: stroke5e
(10000, 1, 105) (10000, 1)


In [5]:

batch = transder_batch(right)
elen, tkn = batch.shape
batch = np.reshape(batch, (elen*tkn))# .shape
Info_Leng = [tk.getGrainTensor(channel, **cs)  for tk in batch]
channelLeng = np.array([info_leng[1] for info_leng in Info_Leng], dtype='float32')
maxGrainLeng = int(np.max(channelLeng))

channelInfo = [info_leng[0] for info_leng in Info_Leng]

channelInfo_Final =np.zeros([len(channelInfo), maxGrainLeng], dtype=int)
for idx, info in enumerate(channelInfo):
    channelInfo_Final[idx, :len(info)] = channelInfo[idx]

channelInfo_Final = channelInfo_Final.reshape(elen, tkn, maxGrainLeng)
channelLeng = channelLeng.reshape(elen, tkn)

print(channelInfo_Final.shape, channelLeng.shape)

(1000, 1, 105) (1000, 1)


# Batch Test

In [32]:

from pprint import pprint
from nlptext.base import BasicObject

CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True, 'Max_Ngram': 1,},
    'char':    {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': True,'Max_Ngram': 3, 'end_grain': True},
    'stroke':  {'use': True,'Max_Ngram': 5, 'end_grain': True},
    # CTX_DEP
    'pos':     {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
    # ANNO
    'annoR':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
    'annoE':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIO',},
}

# Path2Pyramid = 'data/boson/char/Token3870/Pyramid/'
# Path2LGUnique = 'data/boson/char/Token3870/GrainUnique/'



Path2Pyramid  = 'data/LuohuCorpus/char/Token3546/Pyramid/'
Path2LGUnique = 'data/LuohuCorpus/char/Token3546/GrainUnique/'



BasicObject.INIT_FROM_PICKLE(Path2Pyramid, Path2LGUnique)

BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE)

CORPUS	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
FOLDER	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/FOLDER.p
FOLDER	the length of it is   : 44
TEXT	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/TEXT.p
TEXT	the length of it is   : 227876
SENT	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/SENT.p
SENT	the length of it is   : 1213151
TOKEN	read from pickle file : data/LuohuCorpus/char/Token3546/Pyramid/TOKEN.p
TOKEN	the length of it is   : 36760202
**************************************** 

token	read from pickle file : data/LuohuCorpus/char/Token3546/GrainUnique/token.p
token	the length of it is   : 3546
**************************************** 

Deal with the Channel: token
Current Channel is        	 token
Current Channel Max_Ngram 	 1
Deal with the Channel: char
Current Channel is        	 char
Current Channel Max_Ngram 	 1
Deal with the Channel: basic
Current Channel

In [36]:
[i for i in BasicObject.GRAIN_UNI['data/LuohuCorpus/char/Token3546']]

['token',
 'char',
 'basic',
 'medical',
 'radical',
 'subcomp3e',
 'stroke5e',
 'pos']

In [26]:
from nlpembed.utils.batch import generate_batch_idx, transfer_batch, getTokenBatchInfo
from datetime import datetime


maxlen = BasicObject.TOKEN['length']

BATCH = dict(
    skip_window = 5,
    batch_token_num = 1000,
    maxlen = maxlen,
    left2right = 'ctx-tgt', #'tgt-ctx', #
    size2size  = 'All-1' , # 'All-1' (for ctx-tgt only) and '1-All' (for tgt-ctx only)
    tgtInctx   = True,
    # tgt_start_idx = skip_window,
)

if BATCH['size2size'] == '1-1':
    BATCH['batch_size'] = BATCH['batch_token_num'] * (BATCH['skip_window']*2 + int(BATCH['tgtInctx']))
else:
    BATCH['batch_size'] = BATCH['batch_token_num']
    
    
print(BATCH)
tgt_start_idx = BATCH['skip_window']


CH = {'token': {'Max_Ngram': 1},
    #  'char': {'Max_Ngram': 1, 'end_grain': False},
     'basic': {'Max_Ngram': 1, 'end_grain': False},
     'medical': {'Max_Ngram': 1, 'end_grain': False},
     'radical': {'Max_Ngram': 1, 'end_grain': False},
     'subcomp': {'Max_Ngram': 3, 'end_grain': True},
     'stroke': {'Max_Ngram': 5, 'end_grain': True},
      'pos': {'Max_Ngram': 1, 'end_grain': False, 'tagScheme': 'BIO'},
}

num_steps = 10

tgt_start_idx = tgt_start_idx = BATCH['skip_window']
for step in range(1, num_steps + 1):
    T_total_start = datetime.now()
    T_batch_start = T_total_start

    # Step1: generate batch data
    batch_left, batch_right, tgt_start_idx = generate_batch_idx(tgt_start_idx,  **BATCH)
    # print(batch_left)
    # print(batch_right)
    batch_left = transfer_batch(batch_left)
    batch_Left_Channels_Info_Leng = {ch: getTokenBatchInfo(batch_left, ch, **cs) for ch, cs in CH.items()}
    batch_Left_Channels_Info = {ch:v[0] for ch, v in batch_Left_Channels_Info_Leng.items()}
    batch_Left_Channels_Leng = {ch:v[1] for ch, v in batch_Left_Channels_Info_Leng.items()}

    # batch_right = [Token(tk_idx) for tk_idx in batch_right]
    
    ##############################################
    batch_right = transfer_batch(batch_right)
    batch_Right_Channels_Info_Leng = {'token': getTokenBatchInfo(batch_right, 'token')}
    batch_Right_Channels_Info = {ch:v[0] for ch, v in batch_Right_Channels_Info_Leng.items()}
    batch_Right_Channels_Leng = {ch:v[1] for ch, v in batch_Right_Channels_Info_Leng.items()}


#     # Step2: feed the batch data to feed_dict
#     feed_dict = {}

#     for ch in TVC_left:
#         feed_dict[Batch_Left_channelInfos[ch] ] = batch_Left_Channels_Info[ch]
#         feed_dict[Batch_Left_channelLengs[ch] ] = batch_Left_Channels_Leng[ch]
#     for ch in TVC_right:
#         feed_dict[Batch_Right_channelInfos[ch]] = batch_Right_Channels_Info[ch]
#         feed_dict[Batch_Right_channelLengs[ch]] = batch_Right_Channels_Leng[ch]


    T_batch_end = datetime.now()
    print(T_batch_end- T_batch_start)

{'skip_window': 5, 'batch_token_num': 1000, 'maxlen': 36760202, 'left2right': 'ctx-tgt', 'size2size': 'All-1', 'tgtInctx': True, 'batch_size': 1000}
0:00:00.269599
0:00:00.277100
0:00:00.251736
0:00:00.232014
0:00:00.240076
0:00:00.234383
0:00:00.252401
0:00:00.237043
0:00:00.254137
0:00:00.230106


{0: None,
 1: None,
 2: None,
 3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None}

In [53]:
a = {'1': 'a', '2':'b'}

d = {'3':4, **a}
d

{'3': 4, '1': 'a', '2': 'b'}

In [46]:
import multiprocessing

def func(x, **ch):
    print(x*10) 

d = {}
for i in range(10):
    p = multiprocessing.Process(target = func, args = (i,))
    d[i] = p
    p.start()
    
for i, p in d.items():
    
    d[i] = p.join()
    

0
10
20
30
40
80
50
60
70
90


In [27]:
batch_Right_Channels_Info['token'].shape

(1000, 1, 1)

In [28]:
batch_Left_Channels_Info['stroke'].shape

(1000, 11, 75)

In [29]:
batch_Left_Channels_Info['token'].shape

(1000, 11, 1)

In [30]:
batch_right

array([[<tk 酸 >],
       [<tk 、 >],
       [<tk 烧 >],
       [<tk 心 >],
       [<tk , >],
       [<tk 口 >],
       [<tk 干 >],
       [<tk 、 >],
       [<tk 口 >],
       [<tk 苦 >],
       [<tk , >],
       [<tk 无 >],
       [<tk 发 >],
       [<tk 热 >],
       [<tk 、 >],
       [<tk 畏 >],
       [<tk 寒 >],
       [<tk , >],
       [<tk 无 >],
       [<tk 厌 >],
       [<tk 油 >],
       [<tk 等 >],
       [<tk 不 >],
       [<tk 适 >],
       [<tk , >],
       [<tk 为 >],
       [<tk 求 >],
       [<tk 系 >],
       [<tk 统 >],
       [<tk 诊 >],
       [<tk 治 >],
       [<tk 遂 >],
       [<tk 来 >],
       [<tk 我 >],
       [<tk 院 >],
       [<tk 就 >],
       [<tk 诊 >],
       [<tk , >],
       [<tk 由 >],
       [<tk 门 >],
       [<tk 诊 >],
       [<tk 收 >],
       [<tk 住 >],
       [<tk 我 >],
       [<tk 科 >],
       [<tk 。 >],
       [<tk 现 >],
       [<tk 病 >],
       [<tk 史 >],
       [<tk : >],
       [<tk 6 >],
       [<tk 天 >],
       [<tk 前 >],
       [<tk 无 >],
       [<tk 明 >],
       [<t

In [31]:
batch_left

array([[<tk 尤 >, <tk 甚 >, <tk , >, ..., <tk , >, <tk 口 >, <tk 酸 >],
       [<tk 甚 >, <tk , >, <tk 伴 >, ..., <tk 口 >, <tk 干 >, <tk 、 >],
       [<tk , >, <tk 伴 >, <tk 反 >, ..., <tk 干 >, <tk 、 >, <tk 烧 >],
       ...,
       [<tk 动 >, <tk 至 >, <tk 今 >, ..., <tk 冒 >, <tk 、 >, <tk 早 >],
       [<tk 至 >, <tk 今 >, <tk 。 >, ..., <tk 、 >, <tk 发 >, <tk 期 >],
       [<tk 今 >, <tk 。 >, <tk 孕 >, ..., <tk 发 >, <tk 热 >, <tk 无 >]],
      dtype=object)